In [5]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton
import torch
torch.__version__

In [6]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
# fourbit_models = [
#     "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
#     "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
#     "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
#     "unsloth/llama-3-8b-Instruct-bnb-4bit",
#     "unsloth/llama-3-70b-bnb-4bit",
#     "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
#     "unsloth/Phi-3-medium-4k-instruct",
#     "unsloth/mistral-7b-bnb-4bit",
#     "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
# ] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2-7B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Qwen2 patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 4070 SUPER. Max memory: 11.994 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

# 1111

In [7]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 28 layers with 0 QKV layers, 28 O layers and 28 MLP layers.


In [8]:
from datasets import load_dataset
dataset = load_dataset('json', data_files='/train.jsonl', split='train')
print(dataset.column_names)

Generating train split: 0 examples [00:00, ? examples/s]

['instruction', 'input', 'output']


In [9]:
from unsloth import to_sharegpt
dataset = to_sharegpt(
    dataset,
    merged_prompt = "{instruction}[[\nYour input is:\n{input}]]",
    output_column_name = "output",
    conversation_extension = 3, # Select more to handle longer conversations
)

Merging columns:   0%|          | 0/29 [00:00<?, ? examples/s]

Converting to ShareGPT:   0%|          | 0/29 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/29 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/29 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/29 [00:00<?, ? examples/s]

Extending conversations:   0%|          | 0/29 [00:00<?, ? examples/s]

In [10]:
from unsloth import standardize_sharegpt
dataset = standardize_sharegpt(dataset)

Standardizing format:   0%|          | 0/29 [00:00<?, ? examples/s]

In [11]:
print(dataset[0])

{'conversations': [{'content': '中企动力和中企：\nYour input is:\n中企和中企动力是一家公司吗？', 'role': 'user'}, {'content': '‘中企动力”简称‘中企”，是一家成立于1999年的老牌互联网公司', 'role': 'assistant'}, {'content': '中企可以为客户制作以下门户网站：\nYour input is:\n中企能做什么类型的门户网站？', 'role': 'user'}, {'content': '内贸营销，全球营销，通用版本', 'role': 'assistant'}, {'content': '中企Saas商城有以下合作客户：\nYour input is:\n中企商城产品有哪些服务过的客户？', 'role': 'user'}, {'content': '湖南乐秀商贸有限公司、扬州老扬城食品有限公司、海门市创佳纺织品有限公司、珠海市斗门永生隆贸易有限公司、惠州市长布科技农业、扬州兴扬农产品...', 'role': 'assistant'}]}


In [12]:
chat_template = """{SYSTEM}
USER: {INPUT}
ASSISTANT: {OUTPUT}"""
from unsloth import apply_chat_template
dataset = apply_chat_template(
    dataset,
    tokenizer = tokenizer,
    chat_template = chat_template,
    default_system_message = "你是中企动力的客服",
)

Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/29 [00:00<?, ? examples/s]

In [13]:
print(dataset[0])

{'conversations': [{'content': '中企动力和中企：\nYour input is:\n中企和中企动力是一家公司吗？', 'role': 'user'}, {'content': '‘中企动力”简称‘中企”，是一家成立于1999年的老牌互联网公司', 'role': 'assistant'}, {'content': '中企可以为客户制作以下门户网站：\nYour input is:\n中企能做什么类型的门户网站？', 'role': 'user'}, {'content': '内贸营销，全球营销，通用版本', 'role': 'assistant'}, {'content': '中企Saas商城有以下合作客户：\nYour input is:\n中企商城产品有哪些服务过的客户？', 'role': 'user'}, {'content': '湖南乐秀商贸有限公司、扬州老扬城食品有限公司、海门市创佳纺织品有限公司、珠海市斗门永生隆贸易有限公司、惠州市长布科技农业、扬州兴扬农产品...', 'role': 'assistant'}], 'text': '你是中企动力的客服\nUSER: 中企动力和中企：\nYour input is:\n中企和中企动力是一家公司吗？\nASSISTANT: ‘中企动力”简称‘中企”，是一家成立于1999年的老牌互联网公司<|im_end|>\nUSER: 中企可以为客户制作以下门户网站：\nYour input is:\n中企能做什么类型的门户网站？\nASSISTANT: 内贸营销，全球营销，通用版本<|im_end|>\nUSER: 中企Saas商城有以下合作客户：\nYour input is:\n中企商城产品有哪些服务过的客户？\nASSISTANT: 湖南乐秀商贸有限公司、扬州老扬城食品有限公司、海门市创佳纺织品有限公司、珠海市斗门永生隆贸易有限公司、惠州市长布科技农业、扬州兴扬农产品...<|im_end|>'}


In [14]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        # num_train_epochs = 1, # For longer training runs!
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/29 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [15]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4070 SUPER. Max memory = 11.994 GB.
5.764 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 29 | Num Epochs = 20
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 40,370,176


Step,Training Loss


In [17]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

174.0767 seconds used for training.
2.9 minutes used for training.
Peak reserved memory = 8.041 GB.
Peak reserved memory for training = 2.277 GB.
Peak reserved memory % of max memory = 67.042 %.
Peak reserved memory for training % of max memory = 18.984 %.


In [17]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                         # Change below!
    {"role": "user",      "content": "Saas商城能做什么"}]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


为中小企业打造全渠道交易场景：
1、满足企业品牌塑造和在线交易的官网独立商城，支持PC+H5双端呈现;
2、全面支持企业ToG、ToB、ToC三大经营诉求，具备政采对接能力，包
括央采、京华云采、各地省采等;<|im_end|>


In [18]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving
#model.save_pretrained_gguf("model", tokenizer, quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],)

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/vocab.json',
 'lora_model/merges.txt',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

In [19]:
import os
print(os.getcwd())

/home/jovyan


In [3]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
#O#-#                                                                         #-#O=#  #                                                                        3.8% 27.0%              34.3% 46.7%#########                   76.7%########################       93.9%
>>> Nvidia GPU detected.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [20]:
# Save to 8bit Q8_0
if True: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
#if False:
#    model.push_to_hub_gguf(
#        "hf/model", # Change hf to your username!
#        tokenizer,
#        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
#        token = "", # Get a token at https://huggingface.co/settings/tokens
#    )

Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.5G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.54 out of 15.53 RAM for saving.


100%|██████████| 28/28 [00:11<00:00,  2.44it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


Unsloth: Converting qwen2 model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at model into q8_0 GGUF format.
The output location will be ./model/unsloth.Q8_0.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00004.safetensors'
INFO:hf-to-gguf:token_embd.weight,         torch.bfloat16 --> Q8_0, shape = {3584, 152064}
INFO:hf-to-gguf:blk.0.attn_norm.weight,    torch.bfloat16 -->

In [21]:
import subprocess
subprocess.Popen(["ollama", "serve"])
import time
time.sleep(3) # Wait for a few seconds for Ollama to load!

In [22]:
print(tokenizer._ollama_modelfile)

FROM {__FILE_LOCATION__}

TEMPLATE """{{ if .System }}{{ .System }}{{ end }}{{ if .Prompt }}
USER: {{ .Prompt }}{{ end }}
ASSISTANT: {{ .Response }}<|im_end|>"""

PARAMETER stop "<|endoftext|>"
PARAMETER stop "<|im_start|>"
PARAMETER stop "<|im_end|>"
SYSTEM "你是中企动力的客服"


In [23]:
!ollama create tongyiqianwen -f ./model/Modelfile

transferring model data ⠋ transferring model data ⠹ transferring model data ⠸ transferring model data ⠸ transferring model data ⠴ transferring model data ⠴ transferring model data ⠦ transferring model data ⠧ transferring model data ⠏ transferring model data ⠋ transferring model data ⠙ transferring model data ⠙ transferring model data ⠸ transferring model data ⠸ transferring model data ⠼ transferring model data ⠦ transferring model data ⠦ transferring model data ⠧ transferring model data ⠇ transferring model data ⠏ transferring model data ⠋ transferring model data ⠹ transferring model data ⠸ transferring model data ⠼ transferring model data ⠴ transferring model data ⠦ transferring model data ⠦ transferring model data ⠧ transferring model data ⠇ transferring model data ⠋ transferring model data ⠋ transferring model data ⠹ transferring model data ⠸ transferring model data ⠼ transferring model data ⠴ transferring model data ⠴ transferring model data ⠧ transferring model data ⠇ transferring

In [36]:
ls /content/model

config.json                       Modelfile                     unsloth.F16.gguf
generation_config.json            model.safetensors.index.json  unsloth.Q4_K_M.gguf
model-00001-of-00004.safetensors  special_tokens_map.json       unsloth.Q5_K_M.gguf
model-00002-of-00004.safetensors  tokenizer_config.json         unsloth.Q8_0.gguf
model-00003-of-00004.safetensors  tokenizer.json
model-00004-of-00004.safetensors  unsloth.BF16.gguf
